# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [ ]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-it/opus.en-it-train.it
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-it/opus.en-it-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-it/opus.en-it-test.it
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-it/opus.en-it-test.en

--2022-06-22 13:39:13--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-it/opus.en-it-train.it
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60742863 (58M)
Saving to: ‘opus.en-it-train.it’

opus.en-it-train.it 100%[===================>]  57.93M  22.3MB/s    in 2.6s    

2022-06-22 13:39:16 (22.3 MB/s) - ‘opus.en-it-train.it’ saved [60742863/60742863]

--2022-06-22 13:39:16--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-it/opus.en-it-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56911423 (54M)
Saving to: ‘opus.en-it-train.en’

opus.en-it-train.en 100%[===================>]  54.27M  21.4MB/s    in 2.5s    

2022-06-22 13:39:19 (21.4 MB/s) - ‘opus.e

In [ ]:
!pip install tokenizers matplotlib sklearn


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.6 MB 5.0 MB/s 


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

import os
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
from IPython.display import Image
from IPython.core.display import HTML 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
en_sents = open('opus.en-it-train.en').read().splitlines()
it_sents = open('opus.en-it-train.it').read().splitlines()

In [ ]:
len(en_sents)

1000000

In [ ]:
len(it_sents)

1000000

In [ ]:
en_sents[-1]

'Great!'

In [ ]:
it_sents[-1]

'Molto bene.'

In [ ]:
tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_en.train(files=["opus.en-it-train.en"], trainer=trainer_en)

tokenizer_it = Tokenizer(BPE())
tokenizer_it.pre_tokenizer = Whitespace()
trainer_it = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer_it.train(files=["opus.en-it-train.it"], trainer=trainer_it)

In [ ]:
tokenizer_en.save('tokenizer_en')
tokenizer_it.save('tokenizer_it')

In [ ]:
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_it = Tokenizer.from_file("tokenizer_it")

In [ ]:
def encode(text, tokenizer, max_len):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [ ]:
PAD_IDX = tokenizer_it.token_to_id('[PAD]')
PAD_IDX

3

In [ ]:
max_len_en, max_len_it = 50, 50

In [ ]:
X_en = [encode(t, tokenizer_en, max_len_en) for t in en_sents]
X_it = [encode(t, tokenizer_it, max_len_it) for t in it_sents]

In [ ]:
len(X_en)

1000000

In [ ]:
len(X_it)

1000000

In [ ]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_it):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)
        
        self.texts_it = [torch.LongTensor(sent) for sent in texts_it]
        self.texts_it = torch.nn.utils.rnn.pad_sequence(self.texts_it, padding_value=PAD_IDX)

        self.length = len(texts_en)
    
    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_it = self.texts_it[:, index]

        return ids_en, ids_it

In [ ]:
X_en_train, X_en_valid, X_it_train, X_it_valid = train_test_split(X_en, X_it, test_size=0.05)

In [ ]:
training_set = Dataset(X_en_train, X_it_train)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

In [ ]:
valid_set = Dataset(X_en_valid, X_it_valid)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

### Transformer (from [pytorch tutorial](https://pytorch.org/tutorials/beginner/transformer_tutorial.html))

In [ ]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size, 
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
#         print('pos inp')
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
#         print('pos dec')
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
#         print('pos out')
        x = self.generator(outs)
#         print('gen')
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)
# During training, we need a subsequent word mask that will prevent model to look into the future words when making predictions. We will also need masks to hide source and target padding tokens. Below, let’s define a function that will take care of both.

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
from time import time
def train(model, iterator, optimizer, criterion, print_every=500):
    
    epoch_loss = []
    ac = []
    
    model.train()  

    for i, (texts_en, texts_it) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) 
        texts_it = texts_it.T.to(DEVICE) 
        

        texts_it_input = texts_it[:-1, :]
        

        (texts_en_mask, texts_it_mask, 
        texts_en_padding_mask, texts_it_padding_mask) = create_mask(texts_en, texts_it_input)
        logits = model(texts_en, texts_it_input, texts_en_mask, texts_it_mask,
                       texts_en_padding_mask, texts_it_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()
        
        texts_it_out = texts_it[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_it_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
        
        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')
        
    return np.mean(epoch_loss)



def evaluate(model, iterator, criterion):
    
    epoch_loss = []
    epoch_f1 = []

    model.eval()
    for i, (texts_en, texts_it) in enumerate(iterator):
        texts_en = texts_en.T.to(DEVICE) 
        texts_it = texts_it.T.to(DEVICE)
        texts_it_input = texts_it[:-1, :]

        
        (texts_en_mask, texts_it_mask, 
        texts_en_padding_mask, texts_it_padding_mask) = create_mask(texts_en, texts_it_input)
        logits = model(texts_en, texts_it_input, texts_en_mask, texts_it_mask,
                       texts_en_padding_mask, texts_it_padding_mask, texts_en_padding_mask)
        
        texts_it_out = texts_it[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), texts_it_out.reshape(-1))
        epoch_loss.append(loss.item())
        
        
    return np.mean(epoch_loss)

In [ ]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
IT_VOCAB_SIZE = tokenizer_it.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, IT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [ ]:
#torch.save(transformer, 'model')

In [ ]:
#transformer = torch.load('model').to(DEVICE)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from timeit import default_timer as timer
NUM_EPOCHS = 10

losses = []

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)
    
    if not losses:
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    elif val_loss < min(losses):
        print(f'Improved from {min(losses)} to {val_loss}, saving model..')
        torch.save(transformer, 'model')
    
    losses.append(val_loss)
        
    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

Loss: 7.309024249076844;
Loss: 6.658414598941803;
Loss: 6.307060996055603;
Loss: 6.064766675949096;
Loss: 5.880776762390137;
Loss: 5.728973263422648;
Loss: 5.6017643765040805;
Loss: 5.490584200382233;
Loss: 5.391606550746494;
First epoch - 4.363163595199585, saving model..
Epoch: 1, Train loss: 5.345, Val loss: 4.363,            Epoch time=1225.000s
Loss: 4.423740203857422;
Loss: 4.377890146493912;
Loss: 4.3411755757331845;
Loss: 4.300256332039833;
Loss: 4.265555804729462;
Loss: 4.229944823582967;
Loss: 4.196107588699886;
Loss: 4.1643239600658415;
Loss: 4.131228025595347;
Improved from 4.363163595199585 to 3.658487998008728, saving model..
Epoch: 2, Train loss: 4.116, Val loss: 3.658,            Epoch time=1230.150s
Loss: 3.742268993854523;
Loss: 3.72279799413681;
Loss: 3.702446423371633;
Loss: 3.6838565759658812;
Loss: 3.668907503890991;
Loss: 3.6502868983745573;
Loss: 3.6321085269791737;
Loss: 3.615466219186783;
Loss: 3.597787172847324;
Improved from 3.658487998008728 to 3.2695125064

In [ ]:
def translate(text):


    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_it.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_it_mask, 
    texts_en_padding_mask, texts_it_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_it_mask,
                   texts_en_padding_mask, texts_it_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_it.token_to_id('[SEP]'), tokenizer_it.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_it_mask, 
        texts_en_padding_mask, texts_it_padding_mask) = create_mask(input_ids_pad, output_ids_pad)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_it_mask,
                       texts_en_padding_mask, texts_it_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_it.id_to_token(i).replace('##', '') for i in output_ids[1:]]))



In [ ]:
translate('Does this function work?')

'Questa funzione funziona ?'

### BLEU Score

In [ ]:
import nltk

In [ ]:
en_sents_test = open('opus.en-it-test.en').read().lower().splitlines()
it_sents_test = open('opus.en-it-test.it').read().lower().splitlines()

In [ ]:
translations = []

for i in range(len(it_sents_test)):
  translations.append(translate(it_sents_test[i]))

In [ ]:
bleus = []

for i, t in enumerate(translations):
  reference = tokenizer_it.encode(t).tokens
  hypothesis = tokenizer_it.encode(it_sents_test[i]).tokens

bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  ))

In [ ]:
bleus

[0.056676653300637835]


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

Back translation -- обычно используется для малорерсурсных языков в случае недостатка данных для составления больших параллельных корпусов, т.е когда мало данных для одного из языков. 
Модель обучается на существуещих данных, а затем переводится на исходных малоресурсный язык, после чего эти данные добавляются к малоресурсному корпусую
 
 Если бы мы взяли мало текстов на английском, но много на русском, мы могли бы использовать этот метод на данных с семинара: обучить трансформер, перевести тестовую выбрку, добавить эти тексты в тренировочную выборку, а затем обучить новый трансформер.
